In [2]:
%pip uninstall -y tensorflow tensorflow-intel tensorflow-gpu
%pip cache purge
%pip install tensorflow==2.16.1



Found existing installation: tensorflow 2.20.0
Uninstalling tensorflow-2.20.0:
Uninstalling tensorflow-2.20.0:
  Successfully uninstalled tensorflow-2.20.0
  Successfully uninstalled tensorflow-2.20.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Files removed: 919 (9300.4 MB)
Files removed: 919 (9300.4 MB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/589.9 MB ? eta -:--:--Downloading tensorflow-2.16.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (589.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.9/589.9 MB 2.0 MB/s eta 0:00:0000:0100:08
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.9/589.9 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.2 MB ? eta -:--:--Downloading ml_dtypes-0.3.2-cp312-cp312-manylinux_2_17_x86_64.man

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # force CPU only

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.models import Model,Sequential
# layers
from tensorflow.keras.layers import (Input,Conv2D,MaxPooling2D,AveragePooling2D,GlobalAveragePooling2D,Flatten,Dense,Dropout,BatchNormalization,Activation,Add,Concatenate,concatenate,Rescaling,Conv2DTranspose)
#optimizers
from tensorflow.keras.optimizers import Adam,SGD
#callbacks
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
#pretrained models
from tensorflow.keras.applications import ResNet50,VGG16,VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Rescaling
#sklearn
#split
from sklearn.model_selection import train_test_split
#metrics
from sklearn.metrics import accuracy_score,classification_report
# file operations
import glob



2025-12-08 12:16:57.936153: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-08 12:16:58.816188: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-12-08 12:16:58.816188: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
# ============================================
# HELPERS: DATA PIPELINES + MODELS
# ============================================

data_dir = '/home/tazmeen/Documents/CV-final/check/data/images'  # CHANGE as needed
IMG_SIZE = (64, 64)
BATCH_SIZE = 4
SEED = 42

if not os.path.isdir(data_dir):
    raise FileNotFoundError(f"Directory not found: {data_dir}. Set data_dir to your images root (one subfolder per class).")

# ---------- DATA LOADERS ----------

def make_raw_datasets():
    train_raw = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.25,
        subset='training',
        seed=SEED,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        label_mode='int'
    )
    val_raw = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.25,
        subset='validation',
        seed=SEED,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        label_mode='int'
    )
    return train_raw, val_raw

# Normalizers
rescale = Rescaling(1./255)
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess


def make_scratch_ds():
    train_raw, val_raw = make_raw_datasets()
    class_names = train_raw.class_names
    train = train_raw.map(lambda x, y: (rescale(x), y), num_parallel_calls=1)
    val   = val_raw.map(lambda x, y: (rescale(x), y), num_parallel_calls=1)
    return train, val, class_names


def make_resnet_ds():
    train_raw, val_raw = make_raw_datasets()
    class_names = train_raw.class_names
    train = train_raw.map(lambda x, y: (resnet_preprocess(x), y), num_parallel_calls=1)
    val   = val_raw.map(lambda x, y: (resnet_preprocess(x), y), num_parallel_calls=1)
    return train, val, class_names

# ---------- MODELS ----------

def build_scratch_cnn(num_classes):
    return Sequential([
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(*IMG_SIZE, 3)),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        GlobalAveragePooling2D(),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])


def build_resnet_feature_extract(num_classes):
    base = ResNet50(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
    base.trainable = False
    inputs = Input(shape=(*IMG_SIZE, 3))
    x = base(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model


def build_resnet_finetune_last_block(num_classes):
    base = ResNet50(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
    # Freeze all, then unfreeze conv5_* block
    base.trainable = False
    for layer in base.layers:
        if layer.name.startswith('conv5_') or layer.name.startswith('bn5_'):
            layer.trainable = True
    inputs = Input(shape=(*IMG_SIZE, 3))
    x = base(inputs, training=True)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    return model

# ---------- TRAIN/REPORT ----------

def compile_and_train(model, train_ds, val_ds, epochs=1, lr=1e-4, steps_per_epoch=None, validation_steps=None):
    model.compile(optimizer=Adam(lr), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        verbose=1
    )
    return history


In [11]:
# ========== Variant 1: Scratch CNN ==========
train_scratch, val_scratch, class_names = make_scratch_ds()
num_classes = len(class_names)
model_scratch = build_scratch_cnn(num_classes)
compile_and_train(
    model_scratch,
    train_scratch,
    val_scratch,
    epochs=1,              # keep tiny to avoid crashes
    lr=3e-4,
    steps_per_epoch=2,     # limit steps for quick sanity
    validation_steps=2
)

# quick eval
loss, acc = model_scratch.evaluate(val_scratch, verbose=0, steps=1)
print(f"Scratch CNN val acc (1 step): {acc:.3f}")


Found 27 files belonging to 2 classes.
Using 21 files for training.
Using 21 files for training.
Found 27 files belonging to 2 classes.
Using 6 files for validation.
Found 27 files belonging to 2 classes.
Using 6 files for validation.
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 384ms/step - accuracy: 0.3333 - loss: 0.7197 - val_accuracy: 0.3333 - val_loss: 0.6973
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 384ms/step - accuracy: 0.3333 - loss: 0.7197 - val_accuracy: 0.3333 - val_loss: 0.6973
Scratch CNN val acc (1 step): 0.000
Scratch CNN val acc (1 step): 0.000
